In [37]:
# Módulos
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [38]:
# Dados
Produtos = [0, 'Produto 1_1', 'Produto 1_2', 'Produto 2_1', 'Produto 2_2', 'Produto 2_3', 'Produto 3_1','Produto 3_2']
entrega_P = {0: 0, 'Produto 1_1': 10, 'Produto 1_2': 12, 'Produto 2_1': 11, 'Produto 2_2': 13, 
             'Produto 2_3': 17, 'Produto 3_1': 12,'Produto 3_2': 16}
producao_P = {0: 0, 'Produto 1_1': 0.5, 'Produto 1_2': 0.5, 'Produto 2_1': 1.5, 'Produto 2_2': 1.5, 
             'Produto 2_3': 1.5, 'Produto 3_1': 1,'Produto 3_2': 1}
horario_inicio = 9
multa = 30
R = 10000

In [39]:
# Modelo
model = ConcreteModel()

# Conjuntos
model.P = Set(initialize=Produtos)
P = model.P

# Parâmetros
model.p = Param(P, initialize=producao_P, within=NonNegativeReals)
model.d = Param(P, initialize=entrega_P, within=NonNegativeReals)
model.h = Param(initialize=horario_inicio, within=NonNegativeReals)
model.m = Param(initialize=multa, within=NonNegativeReals)
p, d, h, m = model.p, model.d, model.h, model.m

# Variáveis de decisão
model.x = Var(P * P, within=Binary)
model.C = Var(P, within=NonNegativeReals)
model.t = Var(P, within=NonNegativeReals)
model.y = Var(P, within=Binary)
x, C, t, y = model.x, model.C, model.t, model.y

# Função objetivo
model.obj = Objective(sense=minimize,
                      expr = m * sum(y[i] for i in P if i != 0))

# Restrições
# Um sucessor por tarefa
model.r_sucessor = ConstraintList()
for i in P:
    model.r_sucessor.add(expr = sum(x[i, j] for j in P if j != i) == 1)
# Um antecessor por tarefa
model.r_antecessor = ConstraintList()
for j in P:
    model.r_antecessor.add(expr = sum(x[i, j] for i in P if i != j) == 1)
# Tempo de término
model.r_termino = ConstraintList()
model.r_termino.add(expr = C[0] == h)
for i in P:
    for j in P:
        if j != i and j != 0:
            model.r_termino.add(expr = C[j] >= C[i] + p[j] * x[i, j] + R * (x[i, j] - 1))
# Tempos de atraso positivos
model.r_atraso = ConstraintList()
for i in P:
    if i != 0:
        model.r_atraso.add(expr = t[i] >= C[i] - d[i])
# Binário de contagem de tarefas atrasadas
model.r_contagem_atraso = ConstraintList()
for i in P:
    if i != 0:
        model.r_contagem_atraso.add(expr = t[i] <= R * y[i])



In [40]:
# Resolução
solver = SolverFactory('glpk')
resultado = solver.solve(model, tee=False)

In [41]:
# Apresentação de resultados
if (resultado.solver.status == SolverStatus.ok) and (resultado.solver.termination_condition != TerminationCondition.infeasible):
    for i in P:
        for j in P:
            if i != j and value(x[i,j]) == 1:
                print('Produto {} precede o {}.'.format(i, j))
    print()
    for i in P:
        if i != 0:
            if value(y[i]) == 1:
                print('O produto {} terá atraso'.format(i))
    print()
    print('O valor mínimo em multa é de R${:.2f}'.format(value(model.obj)))
else:
    print(resultado.solver.termination_condition)

Produto 0 precede o Produto 1_1.
Produto Produto 1_1 precede o Produto 2_1.
Produto Produto 1_2 precede o Produto 2_2.
Produto Produto 2_1 precede o Produto 1_2.
Produto Produto 2_2 precede o Produto 2_3.
Produto Produto 2_3 precede o Produto 3_2.
Produto Produto 3_1 precede o 0.
Produto Produto 3_2 precede o Produto 3_1.

O produto Produto 3_1 terá atraso

O valor mínimo em multa é de R$30.00
